In [7]:
import logging
from kubernetes import config
from testbed.client.manager import TestbedManager

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

config.load_kube_config()
manager = TestbedManager("testbed-dev", True)

ModuleNotFoundError: No module named 'testbed'

In [2]:
deleted_count = manager.delete_all_testbeds()
print(f"Deleted {deleted_count} testbeds")

INFO:testbed.client.manager:Deleting all testbeds
INFO:testbed.client.manager:Deleted 1 testbeds


Deleted 1 testbeds


In [3]:
# For listing testbeds (summary view)
for testbed in manager.list_testbeds():
    print(f"Testbed ID: {testbed.testbed_id}")
    print(f"Instance ID: {testbed.instance_id}")
    print(f"Pod Phase: {testbed.status.pod_phase}")
    print(f"Testbed Ready: {testbed.status.testbed_ready}")
    print(f"Sidecar Ready: {testbed.status.sidecar_ready}")
    print("---")

    testbed_id = testbed.testbed_id

Testbed ID: sympy-sympy-21847-testbed-lv7ev
Instance ID: sympy__sympy-21847
Pod Phase: Running
Testbed Ready: True
Sidecar Ready: True
---


In [4]:
instance_id = "sympy__sympy-21847"
response = manager.create_testbed(instance_id=instance_id)
testbed_id = response.testbed_id

print(f"Created testbed with id {testbed_id}")

INFO:testbed.client.manager:create_testbed(user: None, instance_id: sympy__sympy-21847)
INFO:testbed.client.manager:create_testbed(user: None, instance_id: sympy__sympy-21847, testbed_id: sympy-sympy-21847-testbed-x7xnq) Job and Service created successfully in namespace testbed-dev.


Created testbed with id sympy-sympy-21847-testbed-x7xnq


In [5]:
from testbed.utils import wait_for_testbed_ready

testbed = wait_for_testbed_ready(manager, testbed_id)

Testbed ID: sympy-sympy-21847-testbed-x7xnq | Phase: Pending | IP: N/A | Testbed: False/unknown | Sidecar: False/unknown | Time: 0s

Testbed ID: sympy-sympy-21847-testbed-x7xnq | Phase: Pending | IP: 4.225.50.70 | Testbed: False/unknown | Sidecar: False/unknown | Time: 30s

In [ ]:
client = manager.create_client(testbed_id, timeout=120)
print("Client created")
client.check_health()

Client created


ERROR:testbed.client.client:Error during ping: HTTPConnectionPool(host='135.225.17.216', port=8000): Max retries exceeded with url: /health (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x79b0b23a47a0>, 'Connection to 135.225.17.216 timed out. (connect timeout=30)'))


False

In [ ]:
from testbed.schema import Prediction
instance_id = "sympy__sympy-21847"

prediction = Prediction(run_id="testing", instance_id=instance_id, patch="")
result = client.run_evaluation(prediction)
print(result)

{'instance_id': 'sympy__sympy-21847', 'patch_applied': True, 'resolved': True, 'tests_status': {'FAIL_TO_FAIL': {'failure': [], 'success': []}, 'FAIL_TO_PASS': {'failure': [], 'success': ['test_monomials']}, 'PASS_TO_FAIL': {'failure': [], 'success': []}, 'PASS_TO_PASS': {'failure': [], 'success': ['test_monomial_count', 'test_monomial_mul', 'test_monomial_div', 'test_monomial_gcd', 'test_monomial_lcm', 'test_monomial_max', 'test_monomial_pow', 'test_monomial_min', 'test_monomial_divides']}}}


In [ ]:
# manager.delete_testbed(testbed_id)